## 🛠️ **Data Loading, Processing and Feature Extraction**  

This notebook processes raw sensor data from the **Open Seizure Database (OSDB)** to extract **metadata**, perform **frequency domain analysis** using **FFT**, and compute statistical and physical features such as skewness, kurtosis, and total distance traveled. The resulting notebook generates the following 4 dataframes ~ df_metadata, df_sensordata, df_sensordata_fft, df_features

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import librosa.display
import pandas as pd
import json
from scipy.stats import skew, kurtosis
import numpy as np
import matplotlib.pyplot as plt


# Load the JSON file
file_path = '../../tests/testData/testDataVisualisation.json'  # Replace with your JSON file path
with open(file_path, 'r') as file:
    raw_json = json.load(file)

In [3]:
# Flatten the JSON and extract relevant fields
flattened_data = []

for attribute in raw_json:
    # Extract user_id
    user_id = attribute.get('userId', None)
    datapoints = attribute.get('datapoints', [])
    subtype = attribute.get('subType', None)
    stype = attribute.get('type', None)
    desc = attribute.get('desc', None)    
    seizureTimes = attribute.get('seizureTimes', [])
    sampleFreq = attribute.get('sampleFreq', 25)
    watchSdName = attribute.get('watchSdName', None)


    for point in datapoints:
        eventId = point.get('eventId', None)
        #hrAlarmActive = attribute.get('hrAlarmActive', None)
        #o2SatAlarmActive = attribute.get('o2SatAlarmActive', None)
        #dataSourceName = attribute.get('dataSourceName', None)
        #watchSdName= attribute.get('watchSdName', None)

        
    # Append the flattened structure
    flattened_data.append({
        'eventId': eventId,
        'userId': user_id,
        'subtype': subtype,
        'type': stype,
        'desc':desc,
        'seizureTimes': seizureTimes,
        'sampleFreq': sampleFreq,
        #"hrAlarmActive": hrAlarmActive, 
        #"o2SatAlarmActive": o2SatAlarmActive, 
        'watchSdName': watchSdName, 
        

    })

    # Create a DataFrame
df_metadata = pd.DataFrame(flattened_data)
df_metadata.to_csv('generatedCsvDatasets/metadata.csv', index=False)
# Display the DataFrame
df_metadata.head()

,eventId,userId,subtype,type,desc,seizureTimes,sampleFreq,watchSdName
0,407,39,Other,Seizure,twisting to left. right arm flapping. left a...,"[-38.0, 76.0]",25,None
1,764,39,Other,Seizure,"kneeling up, looking to right","[-35.0, 75.0]",25,None
2,4924,39,Other,Seizure,on back. left arm flapping,"[-25.0, 60.0]",25,None
3,5483,39,Tonic-Clonic,Seizure,null,"[-45.0, 35.0]",25,GarminSD
4,5486,39,Tonic-Clonic,Seizure,null,"[-15.0, 60.0]",25,GarminSD


In [ ]:
# Load the JSON file
file_path = '../../tests/testData/testDataVisualisation.json'  # Replace with your JSON file path
with open(file_path, 'r') as file:
    raw_json = json.load(file)
    
# Flatten the JSON and extract relevant fields
flattened_data = []

for attribute in raw_json:
    user_id = attribute.get('userId', None)
    seizure_times = attribute.get('seizureTimes', [])
    datapoints = attribute.get('datapoints', [])


    for point in datapoints:
        event_id = point.get('eventId', None)
        hr = point.get('hr', [])
        o2Sat = point.get('o2Sat', [])
        rawData = point.get('rawData', [])
        rawData3D = point.get('rawData3D', [])
        alarmState = point.get('alarmState', None)
        alarmPhrase = point.get('alarmPhrase', None)
        # Append every datapoint as a row
        flattened_data.append({
            'eventId': event_id,
            'userId': user_id,
            'hr': hr,
            'o2Sat': o2Sat,
            'rawData': rawData,
            'rawData3D': rawData3D,
            'seizure_times':seizure_times,
            "alarmState":alarmState,
            "alarmPhrase":alarmPhrase
        })


# Create a DataFrame
df_sensordata = pd.DataFrame(flattened_data)
# Sampling frequency (25 Hz as per your clarification)
sampling_rate = 25  # in Hz

# Define FFT calculation function for each row
def calculate_fft(raw_data):
    # Remove the DC component (mean of the signal)
    raw_data = raw_data - np.mean(raw_data)
    
    # Compute the Fourier Transform (FFT) for the entire signal
    fft_result = np.fft.fft(raw_data)
    
    # Compute the frequencies corresponding to the FFT result
    frequencies = np.fft.fftfreq(len(raw_data), d=1/sampling_rate)
    
    # Compute the magnitude of the FFT (absolute value)
    fft_magnitude = np.abs(fft_result)
    
    # Only consider the positive frequencies (the FFT is symmetric)
    positive_frequencies = frequencies[:len(frequencies)//2]
    positive_fft_magnitude = fft_magnitude[:len(frequencies)//2]
    
    return positive_frequencies, positive_fft_magnitude

# Add a new column for FFT data for all rows in the DataFrame
fft_results = []

for _, row in df_sensordata.iterrows():
    # Extract rawData for the row
    raw_data = np.array(row['rawData'])
    
    # Calculate the FFT for the current row
    positive_frequencies, positive_fft_magnitude = calculate_fft(raw_data)
    
    # Store the result as a list of FFT magnitudes
    fft_results.append(list(positive_fft_magnitude))  # Modify if needed to store specific frequency ranges

# Add the FFT results to the DataFrame as a new column
df_sensordata['FFT'] = fft_results

df_sensordata.to_csv('generatedCsvDatasets/sensordata_fft.csv', index=False)

df_sensordata.head(30)


,eventId,userId,hr,o2Sat,rawData,rawData3D,seizure_times,alarmState,alarmPhrase,FFT
0,407,39,67,-1,"[1496, 1480, 1500, 1492, 1496, 1484, 1500, 149...",[],"[-38.0, 76.0]",0,OK,"[1.2960299500264227e-11, 143.05125737182817, 5..."
1,407,39,67,-1,"[1492, 1508, 1496, 1476, 1484, 1476, 1496, 150...",[],"[-38.0, 76.0]",0,OK,"[9.094947017729282e-13, 75.0235079481899, 31.0..."
2,407,39,68,-1,"[1488, 1496, 1484, 1492, 1492, 1508, 1504, 148...",[],"[-38.0, 76.0]",0,OK,"[2.2737367544323206e-13, 91.25440903139302, 81..."
3,407,39,69,-1,"[1488, 1476, 1480, 1504, 1496, 1508, 1484, 148...",[],"[-38.0, 76.0]",0,OK,"[1.3642420526593924e-11, 101.37768172754973, 7..."
4,407,39,69,-1,"[1504, 1488, 1504, 1492, 1484, 1500, 1496, 149...",[],"[-38.0, 76.0]",0,OK,"[7.275957614183426e-12, 116.42740204040987, 77..."
5,407,39,70,-1,"[1500, 1480, 1496, 1488, 1480, 1488, 1484, 149...",[],"[-38.0, 76.0]",0,OK,"[1.4097167877480388e-11, 64.1735150114216, 146..."
6,407,39,72,-1,"[1500, 1476, 1484, 1468, 1500, 1472, 1496, 148...",[],"[-38.0, 76.0]",0,OK,"[1.8189894035458565e-12, 2033.467961869582, 26..."
7,407,39,72,-1,"[1432, 1304, 1320, 1180, 1308, 1392, 1384, 129...",[],"[-38.0, 76.0]",0,OK,"[1.318767317570746e-11, 8830.291963065638, 253..."
8,407,39,71,-1,"[1504, 1476, 1448, 1592, 1612, 1420, 1348, 166...",[],"[-38.0, 76.0]",0,OK,"[1.0686562745831907e-11, 2977.6280564474355, 6..."
9,407,39,71,-1,"[1532, 1532, 1488, 1516, 1512, 1508, 1536, 156...",[],"[-38.0, 76.0]",0,OK,"[3.410605131648481e-12, 10345.864896884996, 99..."


In [5]:
# Sampling frequency (25 Hz as per your clarification)
sampling_rate = 25  # in Hz

# Define FFT calculation function for each row
def calculate_fft(raw_data):
    # Remove the DC component (mean of the signal)
    raw_data = raw_data - np.mean(raw_data)
    
    # Compute the Fourier Transform (FFT) for the entire signal
    fft_result = np.fft.fft(raw_data)
    
    # Compute the frequencies corresponding to the FFT result
    frequencies = np.fft.fftfreq(len(raw_data), d=1/sampling_rate)
    
    # Compute the magnitude of the FFT (absolute value)
    fft_magnitude = np.abs(fft_result)
    
    # Only consider the positive frequencies (the FFT is symmetric)
    positive_frequencies = frequencies[:len(frequencies)//2]
    positive_fft_magnitude = fft_magnitude[:len(frequencies)//2]
    
    return positive_frequencies, positive_fft_magnitude

# Add a new column for FFT data for all rows in the DataFrame
fft_results = []

for _, row in df_sensordata.iterrows():
    # Extract rawData for the row
    raw_data = np.array(row['rawData'])
    
    # Calculate the FFT for the current row
    positive_frequencies, positive_fft_magnitude = calculate_fft(raw_data)
    
    # Store the result as a list of FFT magnitudes
    fft_results.append(list(positive_fft_magnitude))  # Modify if needed to store specific frequency ranges

# Add the FFT results to the DataFrame as a new column
df_sensordata['FFT'] = fft_results

df_sensordata.to_csv('generatedCsvDatasets/sensordata_fft.csv', index=False)

df_sensordata.head()


,eventId,userId,hr,o2Sat,rawData,rawData3D,seizure_times,FFT
0,407,39,67,-1,"[1496, 1480, 1500, 1492, 1496, 1484, 1500, 149...",[],"[-38.0, 76.0]","[1.2960299500264227e-11, 143.05125737182817, 5..."
1,407,39,67,-1,"[1492, 1508, 1496, 1476, 1484, 1476, 1496, 150...",[],"[-38.0, 76.0]","[9.094947017729282e-13, 75.0235079481899, 31.0..."
2,407,39,68,-1,"[1488, 1496, 1484, 1492, 1492, 1508, 1504, 148...",[],"[-38.0, 76.0]","[2.2737367544323206e-13, 91.25440903139302, 81..."
3,407,39,69,-1,"[1488, 1476, 1480, 1504, 1496, 1508, 1484, 148...",[],"[-38.0, 76.0]","[1.3642420526593924e-11, 101.37768172754973, 7..."
4,407,39,69,-1,"[1504, 1488, 1504, 1492, 1484, 1500, 1496, 149...",[],"[-38.0, 76.0]","[7.275957614183426e-12, 116.42740204040987, 77..."


In [8]:
import pandas as pd
import numpy as np
import json

# Load the JSON file
file_path = '../../tests/testData/testDataVisualisation.json'  # Replace with your JSON file path
with open(file_path, 'r') as file:
    raw_json = json.load(file)

# Flatten the JSON and extract relevant fields
def flatten_json_to_df(raw_json):
    flattened_data = []
    for attribute in raw_json:
        user_id = attribute.get('userId', None)
        seizure_times = attribute.get('seizureTimes', [])
        datapoints = attribute.get('datapoints', [])

        for point in datapoints:
            event_id = point.get('eventId', None)
            hr = point.get('hr', [])
            o2Sat = point.get('o2Sat', [])
            rawData = point.get('rawData', [])
            rawData3D = point.get('rawData3D', [])
            alarmState = point.get('alarmState', None)
            alarmPhrase = point.get('alarmPhrase', None)
            flattened_data.append({
                'eventId': event_id,
                'userId': user_id,
                'hr': hr,
                'o2Sat': o2Sat,
                'rawData': rawData,
                'rawData3D': rawData3D,
                'seizure_times': seizure_times,
                "alarmState": alarmState,
                "alarmPhrase": alarmPhrase
            })

    return pd.DataFrame(flattened_data)

# Create the DataFrame
df_sensordata = flatten_json_to_df(raw_json)

# Sampling frequency (25 Hz as per your clarification)
sampling_rate = 25  # in Hz

# Define FFT calculation function for each row
def calculate_fft(raw_data):
    raw_data = raw_data - np.mean(raw_data)  # Remove the DC component
    fft_result = np.fft.fft(raw_data)  # Compute FFT
    frequencies = np.fft.fftfreq(len(raw_data), d=1/sampling_rate)  # Compute frequencies
    fft_magnitude = np.abs(fft_result)  # Compute the magnitude
    positive_frequencies = frequencies[:len(frequencies)//2]  # Only positive frequencies
    positive_fft_magnitude = fft_magnitude[:len(frequencies)//2]  # Only positive FFT magnitudes
    return positive_frequencies, positive_fft_magnitude

# Add a new column for FFT data for all rows in the DataFrame
def add_fft_column(df):
    fft_results = []
    for _, row in df.iterrows():
        raw_data = np.array(row['rawData'])
        _, positive_fft_magnitude = calculate_fft(raw_data)  # Calculate FFT for the row
        fft_results.append(list(positive_fft_magnitude))  # Append FFT result
    df['FFT'] = fft_results
    return df

# Add the FFT column to the DataFrame
df_sensordata = add_fft_column(df_sensordata)

# Step 1: Add 'timestep' column in 5-second increments
df_sensordata['timestep'] = df_sensordata.index * 5

# Step 2: Add 'label' column, initialized to 0
df_sensordata['label'] = 0

# Step 3: Function to label based on 'seizure_times' and 'alarmPhrase'
def label_alarm_events(df):
    for idx, row in df.iterrows():
        if row['alarmPhrase'] == 'ALARM':  # If alarmPhrase is ALARM
            alarm_time = row['timestep']
            seizure_times = row['seizure_times']
            
            # Process the seizure times list, assuming seizure_times are in seconds
            for seizure in seizure_times:
                start_time = alarm_time + seizure  # Adjust by the seizure offset
                
                # Label the rows before and after the alarm (within the range of seizure_times)
                # For the negative offset (before alarm)
                before_idx = df[(df['timestep'] >= start_time) & (df['timestep'] < alarm_time)].index
                df.loc[before_idx, 'label'] = 1  # Mark as seizure (1)
                
                # For the positive offset (after alarm)
                after_idx = df[(df['timestep'] >= alarm_time) & (df['timestep'] <= start_time)].index
                df.loc[after_idx, 'label'] = 1  # Mark as seizure (1)
    return df

# Apply the function to label the DataFrame
df_sensordata = label_alarm_events(df_sensordata)

# Step 4: Drop the 'seizure_times' column
df_sensordata.drop(columns=['seizure_times','alarmState'], inplace=True)

# Step 5: Save the labeled DataFrame to a CSV file
df_sensordata.to_csv('generatedCsvDatasets/sensordata_labeled.csv', index=False)

# Display the first few rows of the updated DataFrame
df_sensordata.head(3)

,eventId,userId,hr,o2Sat,rawData,rawData3D,alarmPhrase,FFT,timestep,label
0,407,39,67,-1,"[1496, 1480, 1500, 1492, 1496, 1484, 1500, 149...",[],OK,"[1.2960299500264227e-11, 143.05125737182817, 5...",0,0
1,407,39,67,-1,"[1492, 1508, 1496, 1476, 1484, 1476, 1496, 150...",[],OK,"[9.094947017729282e-13, 75.0235079481899, 31.0...",5,0
2,407,39,68,-1,"[1488, 1496, 1484, 1492, 1492, 1508, 1504, 148...",[],OK,"[2.2737367544323206e-13, 91.25440903139302, 81...",10,0


In [9]:
# Sampling frequency (25 Hz as per your clarification)
sampling_rate = 25  # in Hz

# Define FFT calculation function for each row
def calculate_fft(raw_data):
    # Remove the DC component (mean of the signal)
    raw_data = raw_data - np.mean(raw_data)
    
    # Compute the Fourier Transform (FFT) for the entire signal
    fft_result = np.fft.fft(raw_data)
    
    # Compute the frequencies corresponding to the FFT result
    frequencies = np.fft.fftfreq(len(raw_data), d=1/sampling_rate)
    
    # Compute the magnitude of the FFT (absolute value)
    fft_magnitude = np.abs(fft_result)
    
    # Only consider the positive frequencies (the FFT is symmetric)
    positive_frequencies = frequencies[:len(frequencies)//2]
    positive_fft_magnitude = fft_magnitude[:len(frequencies)//2]
    
    return positive_frequencies, positive_fft_magnitude

# Add a new column for FFT data for all rows in the DataFrame
fft_results = []

for _, row in df_sensordata.iterrows():
    # Extract rawData for the row
    raw_data = np.array(row['rawData'])
    
    # Calculate the FFT for the current row
    positive_frequencies, positive_fft_magnitude = calculate_fft(raw_data)
    
    # Store the result as a list of FFT magnitudes
    fft_results.append(list(positive_fft_magnitude))  # Modify if needed to store specific frequency ranges

# Add the FFT results to the DataFrame as a new column
df_sensordata['FFT'] = fft_results
# Sampling frequency (25 Hz)
sampling_rate = 25  # Hz

# Function to calculate total distance for rawData
def calculate_total_distance(raw_data):
    # Convert from milli-g to m/s² (milli-g * 9.81 / 1000)
    raw_data_m_s2 = raw_data * (9.81 / 1000)
    
    # Calculate time step (1 / sampling rate)
    time_step = 1 / sampling_rate
    
    # Calculate distance traveled for each timestep: d = 0.5 * a * t^2
    # We square the time step and multiply by 0.5 times the acceleration
    distance_traveled = 0.5 * raw_data_m_s2 * time_step**2
    
    # Sum the distances for the entire 5-second window (total distance)
    total_distance = np.sum(np.abs(distance_traveled))  # Absolute value to account for movement in any direction
    
    return total_distance

# Apply the function to your DataFrame and create a new 'TotalDistance' column
df_sensordata['TotalDistance'] = df_sensordata['rawData'].apply(lambda x: calculate_total_distance(np.array(x)))

# View the updated DataFrame with the 'TotalDistance' column
df_sensordata.head()


,eventId,userId,hr,o2Sat,rawData,rawData3D,alarmPhrase,FFT,timestep,label,TotalDistance
0,407,39,67,-1,"[1496, 1480, 1500, 1492, 1496, 1484, 1500, 149...",[],OK,"[1.2960299500264227e-11, 143.05125737182817, 5...",0,0,1.460450
1,407,39,67,-1,"[1492, 1508, 1496, 1476, 1484, 1476, 1496, 150...",[],OK,"[9.094947017729282e-13, 75.0235079481899, 31.0...",5,0,1.459916
2,407,39,68,-1,"[1488, 1496, 1484, 1492, 1492, 1508, 1504, 148...",[],OK,"[2.2737367544323206e-13, 91.25440903139302, 81...",10,0,1.461643
3,407,39,69,-1,"[1488, 1476, 1480, 1504, 1496, 1508, 1484, 148...",[],OK,"[1.3642420526593924e-11, 101.37768172754973, 7...",15,0,1.460827
4,407,39,69,-1,"[1504, 1488, 1504, 1492, 1484, 1500, 1496, 149...",[],OK,"[7.275957614183426e-12, 116.42740204040987, 77...",20,0,1.462145


In [6]:
# Function to calculate additional features
def calculate_additional_features(raw_data):
    # Calculate Skewness
    skewness = skew(raw_data)
    
    # Calculate Kurtosis
    kurt = kurtosis(raw_data)
    
    # Calculate Standard Deviation
    std_dev = np.std(raw_data)
    
    # Calculate Maximum Acceleration
    max_acceleration = np.max(np.abs(raw_data))  # Max magnitude of acceleration
    
    return skewness, kurt, std_dev, max_acceleration

# Add new columns for each row in the DataFrame
additional_features = []

for _, row in df_sensordata.iterrows():
    # Extract rawData for the row
    raw_data = np.array(row['rawData'])
       # Calculate additional features
    skewness, kurt, std_dev, max_acceleration = calculate_additional_features(raw_data)
    
    # Store the result as a list of additional features
    additional_features.append([skewness, kurt, std_dev, max_acceleration])

# Convert the list of additional features to a DataFrame
additional_features_df = pd.DataFrame(additional_features, columns=['Skewness', 'Kurtosis', 'StdDev', 'MaxAcceleration'])

# Concatenate the new features with the original DataFrame, making sure there are no duplicates
df_sensordata = pd.concat([df_sensordata.drop(columns=['Skewness', 'Kurtosis', 'StdDev', 'MaxAcceleration'], errors='ignore'), additional_features_df], axis=1)

# Optionally, you can view the DataFrame to confirm the new columns have been added
df_sensordata.to_csv('generatedCsvDatasets/features.csv', index=False)

df_sensordata.head(5)


,eventId,userId,hr,o2Sat,rawData,rawData3D,FFT,TotalDistance,Skewness,Kurtosis,StdDev,MaxAcceleration
0,407,39,67,-1,"[1496, 1480, 1500, 1492, 1496, 1484, 1500, 149...",[],"[1.2960299500264227e-11, 143.05125737182817, 5...",1.460450,0.361479,0.259783,10.162003,1520.0
1,407,39,67,-1,"[1492, 1508, 1496, 1476, 1484, 1476, 1496, 150...",[],"[9.094947017729282e-13, 75.0235079481899, 31.0...",1.459916,-0.154103,-0.599913,9.286288,1508.0
2,407,39,68,-1,"[1488, 1496, 1484, 1492, 1492, 1508, 1504, 148...",[],"[2.2737367544323206e-13, 91.25440903139302, 81...",1.461643,0.029643,-0.209197,10.692694,1516.0
3,407,39,69,-1,"[1488, 1476, 1480, 1504, 1496, 1508, 1484, 148...",[],"[1.3642420526593924e-11, 101.37768172754973, 7...",1.460827,0.497013,0.355193,10.101564,1520.0
4,407,39,69,-1,"[1504, 1488, 1504, 1492, 1484, 1500, 1496, 149...",[],"[7.275957614183426e-12, 116.42740204040987, 77...",1.462145,-0.316861,-0.086010,9.912048,1512.0


## **Step 1: Data Loader** 📂 
**Description**:  
In the first step, the script loads the raw JSON data file, which contains sensor data, event information, and metadata for each user.  
- **Purpose**: To make the data accessible for processing and analysis.  
- **Process**: The JSON file is read and stored into a variable (`raw_json`) for further use.  
- **Input**: A JSON file located at the specified path.  
- **Output**: Raw nested JSON data stored in memory.  

---

## **Step 2: Metadata Processing** 🗂️  
**Description**:  
The second step processes and **flattens the JSON data** to extract essential metadata fields.  
- **Purpose**: Organize relevant details such as `eventId`, `userId`, `type`, `subtype`, `description`, and additional attributes into a structured format.  
- **Process**:  
   - Loops through the JSON data to extract key fields.  
   - Organizes these fields into a tabular structure.  
   - Saves the metadata as a **CSV file** for easy access and storage.  
- **Output**:  
   - A clean DataFrame (`df_metadata`) containing all metadata information.  
   - A CSV file (`metadata.csv`) stored in the `generatedCsvDatasets` directory.  

---

## **Step 3: Sensor Data Processing** ⚙️  
**Description**:  
The third step focuses on extracting and organizing **raw sensor data** (`rawData`) from the JSON file.  
- **Purpose**: Transform time series sensor readings into a format that can be analyzed and visualized.  
- **Process**:  
   - Iterates through the flattened metadata.  
   - Extracts `rawData` for each `eventId`.  
   - Prepares a structured DataFrame for sensor data.  
- **Output**: A DataFrame (`df_sensordata`) with raw sensor readings mapped to corresponding events.

---

## **Step 4: Frequency Domain Analysis (FFT)** 🔍  
**Description**:  
The fourth step applies the **Fast Fourier Transform (FFT)** to the `rawData`.  
- **Purpose**: Analyze the frequency components of the sensor data to uncover signal patterns.  
- **Process**:  
   - Removes the **DC component** (signal mean).  
   - Computes the FFT and retains **positive frequencies** only.  
   - Adds the FFT magnitude results to a new column (`FFT`).  
- **Output**: A new column containing FFT results, enabling frequency domain analysis for each signal.

---

## **Step 5: Feature Extraction** 📈  
**Description**:  
The final step extracts meaningful **statistical and physical features** from the time-domain sensor data.  
- **Purpose**: Generate features that can be used for downstream tasks like analysis, visualization, or machine learning.  
- **Features Extracted**:  
   - **Skewness**: Measure of asymmetry.  
   - **Kurtosis**: Sharpness of the signal distribution.  
   - **Standard Deviation**: Spread of values.  
   - **Max Acceleration**: Maximum signal magnitude.  
   - **Total Distance**: Approximated by integrating acceleration over time.  
- **Output**:  
   - A DataFrame (`df_features`) with the new features added.  
   - A CSV file (`features.csv`) containing the final enriched dataset.

---

# 👤 Author  
Developed for the Open Seizure Database by **Jamie Pordoy**.